In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install trimesh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.8/704.8 kB 41.1 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import sys
sys.path.append("/content/drive/MyDrive/pix3d/byop-2425/models/")
from var_autoencoders import Encoder,Decoder,VAE
import torch
import trimesh
import scipy.io
from torch.utils.data import Dataset
from PIL import Image
import os
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F


In [4]:
#data preparation
json_dir = "/content/drive/MyDrive/pix3d/pix3d.json"
with open(json_dir, 'r') as f:
    data = json.load(f)

In [5]:
df = pd.DataFrame(data)
df.head()

,img,category,img_size,2d_keypoints,mask,img_source,model,model_raw,model_source,3d_keypoints,voxel,rot_mat,trans_mat,focal_length,cam_position,inplane_rotation,truncated,occluded,slightly_occluded,bbox
0,img/bed/0001.png,bed,"[395, 244]","[[[182.5, 147.09375], [174.5, 225.09375], [16....",mask/bed/0001.png,ikea,model/bed/IKEA_MALM_2/model.obj,None,ikea,model/bed/IKEA_MALM_2/3d_keypoints.txt,model/bed/IKEA_MALM_2/voxel.mat,"[[0.7813941591465821, 0.00095539348511137, -0....","[-0.00024347016915001151, 0.09068297313399999,...",35.270398,"[-0.7062195301276326, 0.2367305448542897, -0.8...",-0.078517,False,False,False,"[4, 22, 362, 228]"
1,img/bed/0002.png,bed,"[1007, 599]","[[[309.29285714285714, 543.9148660714286], [-1...",mask/bed/0002.png,ikea,model/bed/IKEA_MALM_2/model.obj,None,ikea,model/bed/IKEA_MALM_2/3d_keypoints.txt,model/bed/IKEA_MALM_2/voxel.mat,"[[0.6331473196939317, 0.08400992130502279, -0....","[0.025652375712099995, 0.0434050556712, 1.1086...",32.378901,"[-0.8365679093356918, 0.3969797870961433, -0.6...",-0.107273,True,True,False,"[46, 47, 927, 599]"
2,img/bed/0003.png,bed,"[372, 292]","[[[308.0, 202.09375], [-1.0, -1.0], [-1.0, -1....",mask/bed/0003.png,ikea,model/bed/IKEA_HEMNES_1/model.obj,None,ikea,model/bed/IKEA_HEMNES_1/3d_keypoints.txt,model/bed/IKEA_HEMNES_1/voxel.mat,"[[0.9337851369875168, 0.004246139989357373, -0...","[0.05160487022685999, 0.0022441725076000067, 0...",39.511348,"[-0.33341418148967134, 0.18052455616467913, -0...",-0.019073,True,False,False,"[0, 33, 344, 292]"
3,img/bed/0004.png,bed,"[1063, 755]","[[[230.06357142857144, 399.5266517857143], [93...",mask/bed/0004.png,ikea,model/bed/IKEA_HEMNES_1/model.obj,None,ikea,model/bed/IKEA_HEMNES_1/3d_keypoints.txt,model/bed/IKEA_HEMNES_1/voxel.mat,"[[0.5029912563246631, 0.09073372306215703, -0....","[0.02463177822849999, 0.0738148517616, 1.09891...",31.871608,"[-0.9464253871842163, 0.2661593054097636, -0.4...",-0.120969,False,False,False,"[89, 74, 984, 738]"
4,img/bed/0005.png,bed,"[414, 449]","[[[-1.0, -1.0], [-1.0, -1.0], [-1.0, -1.0], [4...",mask/bed/0005.png,ikea,model/bed/IKEA_MALM_2/model.obj,None,ikea,model/bed/IKEA_MALM_2/3d_keypoints.txt,model/bed/IKEA_MALM_2/voxel.mat,"[[0.7883156484286317, -0.04024974826747893, 0....","[-0.205655543756, 0.057996630364799975, 1.6363...",89.131102,"[1.1526451872054488, 0.3727803200217579, -1.12...",-0.083036,True,False,True,"[38, 44, 414, 420]"


In [25]:
sample_data_dir = "/content/sample_data.csv"
sample_data = pd.read_csv(sample_data_dir)

In [6]:
class pix3d_dataset(Dataset):
    def __init__(self, dataframe, transform=None, data_dir='/content/drive/MyDrive/pix3d/'):
        self.transform = transform
        self.dataframe = dataframe
        self.data_dir = data_dir

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.data_dir + self.dataframe.iloc[idx]['img']
        mask_path = self.data_dir + self.dataframe.iloc[idx]['mask']
        voxel_path = self.data_dir + self.dataframe.iloc[idx]['voxel']

        image = Image.open(img_path).convert('RGB')
        mask = Image.open(mask_path).convert('L')
        voxel = scipy.io.loadmat(voxel_path)['voxel']

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        voxel = torch.tensor(voxel, dtype=torch.float32).unsqueeze(0)
        sample = {
            'image': image,
            'mask': mask,
            'voxel': voxel
        }
        return sample


In [7]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])
dataset = pix3d_dataset(dataframe=df, transform=transform)
dataloader = DataLoader (dataset, batch_size=1, shuffle=False)

In [10]:
for batch in dataloader:
    images = batch['image']
    # masks = batch['mask']
    voxel = batch['voxel']

    print("Images:", images.shape)
    # print("Masks:", masks.shape)
    # print("Vertices:", vertices.shape)
    # print("Faces:", faces.shape)
    print("Voxels:", voxel.shape)
    break

Images: torch.Size([1, 3, 128, 128])
Voxels: torch.Size([1, 1, 128, 128, 128])


In [ ]:
sample_mesh = trimesh.load("../../model/bed/IKEA_BEDDINGE/model.obj")
mesh = sample_mesh.geometry[list(sample_mesh.geometry.keys())[0]]
vertices = mesh.vertices
faces = mesh.faces
print(vertices.shape, faces.shape)

(114, 3) (72, 3)


In [9]:
epochs = 5
lr = 3e-4
batch_size = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VAE(input_dim=3, hidden_dim=64, latent_dim=128, output_shape=(1, 32, 32, 32)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = torch.nn.BCELoss()

# vae = VAE(3*128*128, 256, 128).to(device)
encoder = model.encoder
decoder = model.decoder
# encoder = encoder(3, 256, 128).to(device)
# Decoder = Decoder(128, 256, 32).to(device)

In [10]:
import tqdm as tqdm
# for epoch in tqdm.tqdm(range(epochs)):
#     for i, batch in enumerate(dataloader):
#         images = batch['image'].to(device)
#         masks = batch['mask'].to(device)
#         vertices = batch['vertices'].to(device)
#         faces = batch['faces'].to(device)

#         optimizer.zero_grad()
#         recon_volumes, mu, logvar = model(images)
#         print(recon_volumes.shape, images.shape)
#         # recon_images = recon_images.view(*images.size())
#         recon_images = recon_volumes.view(images.size(0), images.size(1), images.size(2), images.size(3))
#         loss = model.loss_function(recon_images, images, mu, logvar)

#         loss.backward()
#         optimizer.step()

#         if i % 10 == 0:
#             print('Epoch[{}/{}], Step [{}/{}], Loss: {:.3f}'
#                   .format(epoch + 1, epochs, i + 1, len(dataloader), loss.item()))

def train_vae(model, dataloader, epochs, optimizer, device):
    model.train()
    for epoch in tqdm.tqdm(range(epochs)):
        epoch_loss = 0.0
        for i, batch in enumerate(tqdm.tqdm((dataloader))):
            images = batch['image'].to(device)
            target = batch['voxel'].to(device)
            # target = F.interpolate(target, size=(32, 32, 32), mode='trilinear', align_corners=False)
            optimizer.zero_grad()
            # images = images.view(images.size(0), images.size(1) * images.size(2) * images.size(3))
            recon_output, mu, logvar = model(images)
            # print(recon_output.shape, target.shape)
            loss = model.loss_function(recon_output, target, mu, logvar)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss / len(dataloader):.4f}")

In [11]:
train_vae(model, dataloader, epochs, optimizer, device)

  0%|          | 0/5 [04:36<?, ?it/s]


KeyboardInterrupt: 